In [1]:
import torch

from torchvision import models, transforms, datasets

from torchinfo import summary

In [2]:
batch_size = 128
num_worker = 4
max_learning_rate = 0.1
weight_decay = 1e-5
nesterov=True

num_epoch = 300

In [3]:
model = models.inception_v3(pretrained=True)

/home/k1a22015/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/k1a22015/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
model.eval()
summary(model, input_size=(1, 3, 299, 299))

Layer (type:depth-idx)                   Output Shape              Param #
Inception3                               [1, 1000]                 3,326,696
├─BasicConv2d: 1-1                       [1, 32, 149, 149]         --
│    └─Conv2d: 2-1                       [1, 32, 149, 149]         864
│    └─BatchNorm2d: 2-2                  [1, 32, 149, 149]         64
├─BasicConv2d: 1-2                       [1, 32, 147, 147]         --
│    └─Conv2d: 2-3                       [1, 32, 147, 147]         9,216
│    └─BatchNorm2d: 2-4                  [1, 32, 147, 147]         64
├─BasicConv2d: 1-3                       [1, 64, 147, 147]         --
│    └─Conv2d: 2-5                       [1, 64, 147, 147]         18,432
│    └─BatchNorm2d: 2-6                  [1, 64, 147, 147]         128
├─MaxPool2d: 1-4                         [1, 64, 73, 73]           --
├─BasicConv2d: 1-5                       [1, 80, 73, 73]           --
│    └─Conv2d: 2-7                       [1, 80, 73, 73]           5,

In [5]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(299),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [6]:
trainset = datasets.CIFAR100("./data", train=True, download=True, transform=train_transform)
testset = datasets.CIFAR100("./data", train=False, download=True, transform=test_transform)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size, shuffle=True, num_workers=num_worker, pin_memory=True)
testloader = torch.utils.data.DataLoader(testset, batch_size*2, pin_memory=True, num_workers=0)

In [8]:
model.aux_logits = False

# for parameter in model.parameters():
#     parameter.requires_grad = False

In [9]:
from torch import nn, optim

In [10]:
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 100)
)

In [11]:
model.eval()
summary(model, input_size=(1, 3, 299, 299))

Layer (type:depth-idx)                   Output Shape              Param #
Inception3                               [1, 100]                  3,326,696
├─BasicConv2d: 1-1                       [1, 32, 149, 149]         --
│    └─Conv2d: 2-1                       [1, 32, 149, 149]         864
│    └─BatchNorm2d: 2-2                  [1, 32, 149, 149]         64
├─BasicConv2d: 1-2                       [1, 32, 147, 147]         --
│    └─Conv2d: 2-3                       [1, 32, 147, 147]         9,216
│    └─BatchNorm2d: 2-4                  [1, 32, 147, 147]         64
├─BasicConv2d: 1-3                       [1, 64, 147, 147]         --
│    └─Conv2d: 2-5                       [1, 64, 147, 147]         18,432
│    └─BatchNorm2d: 2-6                  [1, 64, 147, 147]         128
├─MaxPool2d: 1-4                         [1, 64, 73, 73]           --
├─BasicConv2d: 1-5                       [1, 80, 73, 73]           --
│    └─Conv2d: 2-7                       [1, 80, 73, 73]           5,

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=max_learning_rate, momentum=0.9, weight_decay=weight_decay, nesterov=nesterov)

In [13]:
model.train()
model = model.cuda()

In [14]:
import time
from tqdm import tqdm

import torch.nn.functional as F
import os

In [15]:
best_loss = 1e10
validation_loss = 1e10
early_stopping = -1
best_acc = 0

count_stop_epoch = 0

model_save_path = os.path.join('./save', 'iception_v3')

train_losses = []
valid_losses = []
train_acc_list = []
valid_top1_list = []
valid_top5_list = []
valid_superclass_list = []
lrs = []

class_to_superclass = {
    4: 0, 30: 0, 55: 0, 72: 0, 95: 0,
    1: 1, 32: 1, 67: 1, 73: 1, 91: 1, 
    54: 2, 62: 2, 70: 2, 82: 2, 92: 2, 
    9: 3, 10: 3, 16: 3, 28: 3, 61: 3, 
    0: 4, 51: 4, 53: 4, 57: 4, 83: 4,
    22: 5, 39: 5, 40: 5, 86: 5, 87: 5,
    5: 6, 20: 6, 25: 6, 84: 6, 94: 6,
    6: 7, 7: 7, 14: 7, 18: 7, 24: 7,
    3: 8, 42: 8, 43: 8, 88: 8, 97: 8,
    12: 9, 17: 9, 37: 9, 68: 9, 76: 9,
    23: 10, 33: 10, 49: 10, 60: 10, 71: 10,
    15: 11, 19: 11, 21: 11, 31: 11, 38: 11,
    34: 12, 63: 12, 64: 12, 66: 12, 75: 12,
    26: 13, 45: 13, 77: 13, 79: 13, 99: 13,
    2: 14, 11: 14, 35: 14, 46: 14, 98: 14,
    27: 15, 29: 15, 44: 15, 78: 15, 93: 15,
    36: 16, 50: 16, 65: 16, 74: 16, 80: 16,
    47: 17, 52: 17, 56: 17, 59: 17, 96: 17,
    8: 18, 13: 18, 48: 18, 58: 18, 90: 18,
    41: 19, 69: 19, 81: 19, 85: 19, 89: 19
}

torch.cuda.empty_cache()
start_time = time.time()

def save_best_model(type):
        torch.save(model.state_dict(), os.path.join(model_save_path, f'{type}_bset.pt'))
        print(f'{type} best model save')
        
def load_model(self, type):
    return torch.load(os.path.join(model_save_path, f'{type}_bset.pt'))

def get_eval(data_loader, get_top5=True, get_superclass=True) :
        top1_pred = 0
        top5_pred = 0
        superclass_pred = 0
        running_loss = 0
        n = 0
        with torch.no_grad() :
            model.eval()
            for X, y_true in tqdm(data_loader, desc='val'):
                X = X.to('cuda')
                y_true = y_true.to('cuda')
                
                y_hat = model(X)
                y_prob = F.softmax(y_hat, dim=1)
                _, predicted_labels = torch.max(y_prob,1)
                
                loss = criterion(y_hat, y_true)
                running_loss += loss.item() * X.size(0)
                
                n += y_true.size(0)
                top1_pred += (predicted_labels == y_true).sum()
                
                if get_top5:
                    _, top5_pred_label = torch.topk(y_prob, 5, dim=1)
                    y_true_expanded = y_true.unsqueeze(1)
                    top5_pred += (top5_pred_label == y_true_expanded).any(dim=1).sum()

                if get_superclass:
                    y_true_super = torch.tensor([class_to_superclass[y.item()] for y in y_true])
                    pred_super = torch.tensor([class_to_superclass[y.item()] for y in predicted_labels])
                    superclass_pred += (y_true_super == pred_super).sum()
        
        epoch_loss = running_loss / len(data_loader.dataset)
        top1 = top5 = superclass = 0
        top1 = top1_pred.float() / n
        if get_top5:
            top5 = top5_pred.float() / n
        if get_superclass:
            superclass = superclass_pred.float() / n
        return top1, top5, superclass, epoch_loss

def feed(train_loader, model, criterion, optimizer):
    model.train()
    running_loss = 0
    
    for idx, (X, y_true) in tqdm(enumerate(train_loader), desc='train'):
        X = X.to('cuda')
        y_true = y_true.to('cuda')
        
        optimizer.zero_grad()
        
        y_hat  = model(X)
        loss = criterion(y_hat, y_true)
        loss.backward()
        
        # nn.utils.clip_grad_value_(self.model.parameters(), self.gradient_clip)   
        optimizer.step()
        
        running_loss += loss.item() * X.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    
    # lr = self.__get_lr()
    # self.lr_scheduler.step()
        
    # print(f'lr: {lr}')
    
    return model , optimizer, epoch_loss

for epoch in range(0, num_epoch):

    model, optimizer, train_loss = feed(trainloader, model, criterion, optimizer)
    train_losses.append(train_loss)

    train_top1, _, _, _ = get_eval(trainloader, get_top5=False, get_superclass=False)
    val_top1, val_top5, val_superclass, valid_loss = get_eval(testloader, get_top5=False, get_superclass=False)
    valid_losses.append(valid_loss)

    train_acc_list.append(train_top1)
    valid_top1_list.append(val_top1)
    valid_top5_list.append(val_top5)
    valid_superclass_list.append(val_superclass)

    print(f'Epoch: {epoch}\t'
        f'Train loss: {train_loss:.4f}\t'
        f'Valid loss: {valid_loss:.4f}\t'
        f'Train accuracy: {100 * train_top1:.2f}\t'
        f'Valid top1 accuracy: {100 * val_top1:.2f}\t'
        #   f'Valid top5 accuracy: {100 * val_top5:.2f}\t'
        #   f'Valid superclass accuracy: {100 * val_superclass:.2f}'
    )
    
    if best_acc < val_top1:
        best_acc = val_top1
        save_best_model('acc')
        print(f'best acc model: epoch {epoch}')
    if best_loss > valid_loss:
        best_loss = valid_loss
        count_stop_epoch = 0
        save_best_model('loss')
        print(f'best loss model: epoch {epoch}')
    else:
        count_stop_epoch += 1
        if early_stopping != -1:
            print(f'{early_stopping - count_stop_epoch} left')
            if count_stop_epoch == early_stopping:
                print(f"Early Stopping")
                break

end_time = time.time()
time_diff = end_time - start_time
save_best_model('last')
print(f'training time: {time_diff // 3600}h {time_diff % 3600 // 60}m {time_diff % 3600 % 60}')

train: 391it [02:15,  2.89it/s]
val: 100%|██████████| 40/40 [00:19<00:00,  2.07it/s]


Epoch: 0	Train loss: 3.4597	Valid loss: 2.4603	Train accuracy: 29.10	Valid top1 accuracy: 35.61	
acc best model save
best acc model: epoch 0
loss best model save
best loss model: epoch 0


train: 391it [02:14,  2.91it/s]
val: 100%|██████████| 40/40 [00:18<00:00,  2.19it/s]


Epoch: 1	Train loss: 2.4602	Valid loss: 1.9191	Train accuracy: 40.92	Valid top1 accuracy: 48.81	
acc best model save
best acc model: epoch 1
loss best model save
best loss model: epoch 1


train: 391it [02:14,  2.92it/s]
val: 100%|██████████| 40/40 [00:18<00:00,  2.22it/s]


...

Epoch: 54	Train loss: 0.6107	Valid loss: 1.0047	Train accuracy: 85.70	Valid top1 accuracy: 78.01	



train: 391it [02:13,  2.92it/s]
val: 100%|██████████| 40/40 [00:17<00:00,  2.24it/s]

Epoch: 55	Train loss: 0.6035	Valid loss: 1.0723	Train accuracy: 85.19	Valid top1 accuracy: 77.36	



train: 33it [00:11,  2.75it/s]


KeyboardInterrupt: 